# Analisys of 2015 survey of Mobility


In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import community

#### Testing section

#### Import Dataset 

In [42]:
G = nx.DiGraph()
file = open('encuesta.csv')
i=0
for l in file:
    ##To skip the header
    if i==0:
        i+=1
        continue
    #print(i, end=" ")
    ### To visualize the import
    #if i>=100:
        #break
    i+=1  
    linea= l.strip().split(',')
    #print(linea[12], end=" ")##origin
    #print(linea[4])
    origen=""
    #print(linea[10].strip(), end=" ")
    if linea[10].strip().startswith("BICICLETA CON MOTOR"):
        origen=linea[13]
    else:
        origen= linea[12]
    #print(origen, end=" ")   
    #print(linea[4])
    #if  origen.startswith('SOACHA') and origen==linea[4] :
        #print(i)
    if (origen, linea[4]) in G.edges():
        G[origen][linea[4]]['weight']+=1
    else:
        G.add_edge(origen, linea[4], weight=1 )
    

##### Checking graph

In [43]:
#print(G.edges(data=True))
#print(G.nodes())

#### Export graph to gephi

In [39]:
nx.write_gexf(G, "survey_mobility_cities.gexf")

#### Make graph of cities

#### make graph of every sector given (ZAT origen and destination)

In [72]:
Gsector = nx.DiGraph()
file = open('encuesta.csv')
i=0
for l in file:
    ##To skip the header
    if i==0:
        i+=1
        continue
    #print(i, end=" ")
    ### To visualize the import
    #if i>=10000:
        #break
    i+=1  
    linea= l.strip().split(',')
    #print(linea[12], end=" ")##origin
    #print(linea[4])
    origen=""
    #print(linea[10].strip(), end=" ")
    if linea[10].strip().startswith("BICICLETA CON MOTOR"):
        origen=linea[11].strip()
        destination=linea[12].strip()
    else:
        origen=linea[10].strip()
        destination=linea[11].strip()
    #print(origen, end=" ")   
    #print(destination)
    if  origen.startswith('904') and destination.startswith('904') :
        print(i, end=" ")
        print(origen, end=" ")   
        print(destination)
    if (origen, destination) in Gsector.edges():
        Gsector[origen][destination]['weight']+=1
    else:
        Gsector.add_edge(origen, destination, weight=1 )
 

66483 904 904
66484 904 904
66485 904 904
66486 904 904
71427 904 904
71428 904 904
71535 904 904
71536 904 904
73090 904 904
73091 904 904
73166 904 904
73167 904 904
73168 904 904
73169 904 904
73170 904 904
73171 904 904
73172 904 904
73173 904 904
73174 904 904
73175 904 904
73178 904 904
73179 904 904
73186 904 904
73187 904 904
73190 904 904
73191 904 904
73192 904 904
73193 904 904
73194 904 904
73195 904 904
73196 904 904
73197 904 904
73198 904 904
73199 904 904
73200 904 904
73201 904 904
73202 904 904
73203 904 904
73229 904 904
73230 904 904
73231 904 904
73232 904 904
73233 904 904
73234 904 904
73235 904 904
73236 904 904
73237 904 904
73238 904 904
73239 904 904
73240 904 904
73241 904 904
73242 904 904
73251 904 904
73252 904 904
73253 904 904
73254 904 904
73255 904 904
73256 904 904
73257 904 904
73258 904 904
73259 904 904
73260 904 904
73261 904 904
73262 904 904
73263 904 904
73264 904 904
73265 904 904
73288 904 904
73289 904 904
73290 904 904
73291 904 904
73294 

In [57]:
#print(Gsector.edges(data=True))
print(Gsector.nodes())

['238', '566', '564', '138', '227', '239', '385', '546', '548', '534', '570', '842', '850', '288', '533', '590', '308', '411', '40', '596', '522', '27', '290', '413', '275', '291', '1000', '340', '193', '993', '565', '523', '418', '417', '410', '255', '849', '130', '241', '900', '557', '396', '412', '87', '527', '552', '951', '554', '163', '544', '528', '170', '420', '459', '454', '446', '532', '932', '322', '276', '588', '702', '542', '393', '442', '390', '269', '120', '207', '386', '289', '563', '423', '517', '535', '580', '44', '964', '365', '389', '419', '254', '310', '545', '547', '429', '560', '536', '328', '530', '199', '112', '104', '41', '62', '171', '108', '339', '105', '841', '114', '350', '462', '540', '416', '342', '336', '256', '88', '902', '375', '456', '96', '180', '107', '209', '167', '115', '444', '37', '443', '33', '26', '779', '502', '89', '503', '501', '280', '691', '992', '606', '500', '689', '602', '475', '616', '755', '86', '235', '332', '301', '473', '300', '24

In [61]:
nx.write_gexf(Gsector, "survey_mobility_sectors.gexf")